In [0]:
import tensorflow as tf
import numpy as np
import PIL
import skimage.io
import vgg19

In [0]:
def display_image(image):
    image = np.squeeze(image)
    image = image[:,:,(2,1,0)]
    image = image + vgg19.VGG_MEAN
    image = np.clip(image, 0, 255).astype(np.uint8)
    display(PIL.Image.fromarray(image))

In [0]:
def load_image(file_style, file_content):
    image_style = (skimage.io.imread(file_style) - vgg19.VGG_MEAN)[:,:,(2,1,0)]
    image_content = (skimage.io.imread(file_content) - vgg19.VGG_MEAN)[:,:,(2,1,0)]
    return image_style[np.newaxis, :, :, :], image_content[np.newaxis, :, :, :]

In [0]:
def gram_matrix(tensor):
    shape = tensor.get_shape()
    channels = shape[3].value
    size = np.prod(shape).value
    matrix = tf.reshape(tensor, shape=[-1, channels])
    return tf.matmul(tf.transpose(matrix), matrix) / size

In [0]:
def get_loss(var, image_model, selection):
    if selection == 0:
        losses = [tf.nn.l2_loss(var1 - var2) for (var1, var2) in zip(var, get_content(image_model))]
    elif selection == 1:
        losses = [tf.nn.l2_loss(var1 - var2) for (var1, var2) in zip(var, get_style(image_model))]
    losses = [losses[i] / (var[i].size) for i in range(len(losses))]
    overall_loss = tf.reduce_sum(losses)
    return overall_loss

In [0]:
def get_style(model):
    tensor = [gram_matrix(getattr(model, layer)) for layer in style_layers]
    return tensor

In [0]:
def get_content(vgg):
    tensor = [getattr(vgg, layer) for layer in content_layers]
    return tensor

In [0]:
def style_transfer(style_file, content_file, n_iter, style_wt, content_wt, vgg_file):
  
    style_img, content_img = load_image(style_file, content_file)
    with tf.Session() as sess:
      
        content_model = vgg19.Vgg19(vgg_file)
        content_vars = tf.placeholder("float", content_img.shape)
        content_model.build(content_vars)
        
        style_model = vgg19.Vgg19(vgg_file)
        style_vars = tf.placeholder("float", style_img.shape)
        style_model.build(style_vars)

        sess.run(tf.global_variables_initializer())
        style = sess.run(get_style(style_model), feed_dict={style_vars: style_img})
        content = sess.run(get_content(content_model), feed_dict={content_vars: content_img})
        
    # start with white noise
    noise = tf.truncated_normal(content_img.shape, stddev=0.1*np.std(content_img))
    mixed_img = tf.Variable(noise)
    mixed_model = vgg19.Vgg19(vgg_file)
    mixed_model.build(mixed_img)

    # define losses and optimizer
    content_loss = get_loss(content, mixed_model, 0)
    style_loss = get_loss(style, mixed_model, 1)
    total_loss = (content_wt * content_loss) + (style_wt * style_loss)
    
    optimizer = tf.train.AdamOptimizer(1.0)
    optimizer = optimizer.minimize(total_loss)

    # perform style transfer
    with tf.Session() as sess:
        print()
        sess.run(tf.global_variables_initializer())
        for i in range(n_iter):
            sess.run(optimizer)
            if (i+1) % 100 == 0:
                print('result for iteration ', (i+1))
                print('content_loss: ', (content_wt * content_loss.eval())) 
                print('style_loss: ', (style_wt * style_loss.eval()))
                display_image(mixed_img.eval())
        print('Done')

In [0]:
content_layers = ['conv4_2']
style_layers = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']

In [0]:
style_transfer('style.jpg', 'content.jpg', n_iter=1000, style_wt=50.0, content_wt=1.0, vgg_file='vgg19.npy') # change weights for varying effects